<a href="https://colab.research.google.com/github/advik-7/HackAurora/blob/main/Gen_RAG_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community -qq
!pip install pypdf -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.6 MB/s eta 0:00:00


In [2]:
!pip install --upgrade google-generativeai -qq

In [3]:

!pip install faiss-cpu==1.7.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.8 MB/s eta 0:00:00


In [5]:
!pip install pypdf

In [6]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langchain.llms.base import LLM
from langchain.vectorstores import FAISS
import requests
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import google.generativeai as genai
import os
from typing import Optional, List, Mapping, Any

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA



In [8]:
def create_faiss_db(all_documents, model_name, faiss_index_path, text_file_path):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1250, chunk_overlap=0)
    texts = text_splitter.split_documents(all_documents)

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode([t.page_content for t in texts])

    class CustomEmbeddings:
        def __init__(self, model):
            self.model = model

        def embed_documents(self, texts):
            return self.model.encode(texts)

        def embed_query(self, text):
            return self.model.encode(text)

        def __call__(self, text):
            return self.model.encode(text)

    custom_embeddings = CustomEmbeddings(model)

    db = FAISS.from_embeddings(
        text_embeddings=[(t.page_content, embedding) for t, embedding in zip(texts, embeddings)],
        embedding=custom_embeddings
    )

    faiss.write_index(db.index, faiss_index_path)

    # Save chunks to text file
    with open(text_file_path, "w", encoding="utf-8") as file:
        for chunk in [t.page_content for t in texts]:
            file.write(chunk + "\n")
@tool
def retrieve_relevant_chunks(query, faiss_index_path, text_file_path, model_name, top_k=5):
    """Generalized function to retrieve relevant chunks using FAISS."""
    index = faiss.read_index(faiss_index_path)
    with open(text_file_path, "r", encoding="utf-8") as file:
        chunks = file.readlines()

    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i].strip() for i in indices[0]]


@tool
def query_llm(prompt: str, context: str = "") -> str:
    """LLM query tool for response generation using Gemini."""
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    model = genai.GenerativeModel("gemini-1.5-flash")
    complete_prompt = f"{context}\n\nQuestion: {prompt}\nAnswer:"
    response = model.generate_content([complete_prompt])
    return response.text

In [16]:


def run_agentic_rag(file_paths, query, model_name="sentence-transformers/LaBSE", top_k=5):
    """Runs Agentic RAG on multiple files and a query."""
    faiss_index_path = "combined_faiss_index.bin" # Use a common index
    text_file_path = "combined_texts3.txt" # Use a common text file
    '''    # Combine documents from all files
    all_documents = []
    for file_path in file_paths:
        loader = PyPDFLoader(file_path)
        all_documents.extend(loader.load())

    # Create the FAISS index using combined documents
    create_faiss_db(all_documents, model_name, faiss_index_path, text_file_path)'''
    tools = [
        Tool(
            name="Main Index Retrieval",
            func=lambda query: retrieve_relevant_chunks(
                query,
                faiss_index_path,
                text_file_path,
                model_name,
                top_k=top_k
            ),
            description="Useful for retrieving relevant chunks from the main index based on a query.",
        ),
        Tool(
            name="Query LLM",
            func=query_llm,
            description="Useful for querying the LLM for response generation.",
        ),
    ]

    genai.configure(api_key=os.environ["GEMINI_API_KEY"])

    class CustomGemini:
        """Custom class to interact with Google Gemini."""

        def __init__(self, temperature: float, max_tokens: int, model: str, google_api_key: str):
            self.temperature = temperature
            self.max_tokens = max_tokens
            self.model = model
            genai.configure(api_key=google_api_key)
            self.generation_config = {
                "temperature": temperature,
                "max_output_tokens": max_tokens,
            }
            self.model_instance = genai.GenerativeModel(
                model_name=model,
                generation_config=self.generation_config,
            )

        def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            response = self.model_instance.generate_content([prompt])
            return response.text
    custom_gemini = CustomGemini(
        temperature=0.7,
        max_tokens=512,
        model="gemini-1.5-flash",
        google_api_key=os.environ["GEMINI_API_KEY"]
    )


    class CustomLLMWrapper(LLM):
        custom_llm: CustomGemini

        @property
        def _llm_type(self) -> str:
            return "custom_gemini"

        def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            return self.custom_llm(prompt, stop=stop)

        @property
        def _identifying_params(self) -> Mapping[str, Any]:
            return {
                "temperature": self.custom_llm.temperature,
                "max_tokens": self.custom_llm.max_tokens,
                "model": self.custom_llm.model,
            }


    wrapped_llm = CustomLLMWrapper(custom_llm=custom_gemini)


    agent = initialize_agent(
        tools=tools,
        llm=wrapped_llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        output_parser=parser,
        handle_parsing_errors=True
    )
      # Run the agent and get the response
    response = agent.run(query)

    # Format the output
    print("**Query:**", query)
    print("\n**Response:**")
    print(response)

In [11]:
from langchain_core.exceptions import OutputParserException  # Assuming langchain_core is used
from langchain_core.output_parsers import BaseOutputParser

class PassthroughOutputParser(BaseOutputParser[str]):  # Inherit from BaseOutputParser
    """Custom output parser that simply returns the raw text."""

    def parse(self, text: str) -> str:
        """Return the raw text as is."""
        return text

    @property
    def _type(self) -> str:
        return "passthrough_output_parser"

In [12]:
parser = PassthroughOutputParser()

In [17]:
run_agentic_rag(["/content/Personal Finance for Dummies.pdf","/content/The Total Money Makeover - Dave Ramsey.pdf"], "i just started earning money , how do i plan my budget nd save money ", model_name="sentence-transformers/LaBSE", top_k=10)



> Entering new AgentExecutor chain...
Thought: I need to provide advice on budgeting and saving money for someone who has just started earning.  I can use the LLM to generate a response that covers key aspects of budgeting and saving.

Action: Query LLM
Action Input: "Create a concise guide for someone who just started earning money on how to plan a budget and save money.  The guide should cover these points: 1. Tracking income and expenses, 2. Creating a budget (including needs vs. wants), 3. Setting savings goals, 4.  Automating savings, 5.  Finding ways to reduce expenses, and 6.  Building an emergency fund."

Observation:

Creating a Budget and Saving Money: A Beginner's Guide

Congratulations on starting to earn money!  Managing your finances effectively is crucial for building a secure future.  Here's a simple guide to get you started:

1. **Track Income and Expenses:**  For at least a month, meticulously track every penny you earn and spend.  Use a budgeting app, spreadsheet, 